In [1]:
import numpy as np
import pandas as pd
import warnings
import time
from scipy import signal
from scipy.signal import butter, lfilter
import matplotlib.pyplot as plt
import math
import pickle
import statistics
import random
from sklearn.metrics.pairwise import cosine_similarity
from math import log
from sklearn import preprocessing
from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
import sktime as sktime
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sktime.datasets import load_from_tsfile_to_dataframe
from sktime.alignment.dtw_python import AlignerDTWfromDist
from sktime.dists_kernels.scipy_dist import ScipyDist
from sktime.dists_kernels.compose_from_align import DistFromAligner
import os
import sys
%matplotlib inline
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import RidgeClassifierCV
from sktime.transformations.panel.rocket import Rocket
from sktime.classification.distance_based import KNeighborsTimeSeriesClassifier
from sktime.classification.hybrid import HIVECOTEV2
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sktime.classification.dictionary_based import MUSE

/Users/abhidnyapatharkar/opt/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


Dictionary saved to Final_hb_gait_MUSE_normalized_cv.pickle
Dictionary saved to Final_healthy_gait_MUSE_normalized_cv.pickle


### Load normal and abnormal datasets

In [2]:
final_normal_df = pd.read_pickle('normal_gait.pkl')
final_normal_df = final_normal_df.drop(columns='time')
final_abnormal_df = pd.read_pickle('abnormal_gait.pkl')
final_abnormal_df = final_abnormal_df.drop(columns='time')

In [3]:
abnormal_indices = (final_abnormal_df['Subject']).unique()
abnormal_no = (final_abnormal_df['Subject']).nunique()

In [4]:
# Specify the file path of the saved pkl file
file_path = 'final_bins_1.pkl'

# Load the dictionary from the pkl file
with open(file_path, 'rb') as file:
    final_numbers_dictionary = pickle.load(file)
# Now, 'loaded_dictionary' contains the dictionary loaded from the pkl file
display("Dictionary loaded from pickle file:")

'Dictionary loaded from pickle file:'

{'sample_numbers_list_0.0_to_0.1': [42, 45, 11, 21, 47, 49],
 'sample_numbers_list_0.1_to_0.2': [12, 70, 13, 38, 20, 31, 44, 57, 58],
 'sample_numbers_list_0.2_to_0.3': [36, 2, 6, 62, 26],
 'sample_numbers_list_0.3_to_0.4': [5, 17, 37, 56],
 'sample_numbers_list_0.4_to_0.5': [18, 3, 8, 43, 52],
 'sample_numbers_list_0.5_to_0.6': [40, 71],
 'sample_numbers_list_0.6_to_0.7': [14, 72, 46, 27, 1, 9, 23],
 'sample_numbers_list_0.7_to_0.8': [15, 4, 32, 33, 59, 25, 54],
 'sample_numbers_list_0.8_to_0.9': [67, 7, 16, 48, 53, 63],
 'sample_numbers_list_0.9_to_1.0': [19, 64],
 'sample_numbers_list_1.0_to_1.1': [10, 30, 34, 61],
 'sample_numbers_list_1.1_to_1.2': [28, 60, 41, 24],
 'sample_numbers_list_1.2_to_1.3': [51],
 'sample_numbers_list_1.3_to_1.4': [29, 50],
 'sample_numbers_list_1.4_to_1.5': [68, 22],
 'sample_numbers_list_1.5_to_1.6': [55, 39],
 'sample_numbers_list_1.6_to_1.7': [],
 'sample_numbers_list_1.7_to_1.8': [65],
 'sample_numbers_list_1.8_to_1.9': [],
 'sample_numbers_list_1.9_

### Function to find the length of the time series

In [5]:
# Define a function to z-normalize a column
def z_normalize(column):
    mean = column.mean()
    std = column.std()
    z_normalized = (column - mean) / std
    return z_normalized

In [6]:
def time_series_length_finding(df):
    train_1_abnormal = df.copy()
    train_1_ab_sub = (train_1_abnormal['Subject'].unique()).tolist()
    row_count = []
    for i in train_1_ab_sub:
        train_1_abnormal_new = train_1_abnormal[(train_1_abnormal['Subject']== i)]
        no_rows = train_1_abnormal_new.shape[0]
        row_count.append(no_rows)
#     display("Max length:",max(row_count))
#     display("Min length:",min(row_count))
    display("Avg length:",statistics.mean(row_count))
    avg_len_ts = int(statistics.mean(row_count))
#     display("Median length:",statistics.median(row_count))
    return(avg_len_ts)

In [7]:
def preprocessing_normal_df(df,desired_length):
    train_2_normal = df.copy()
    df_train_3 = pd.DataFrame(columns = train_2_normal.columns)
    num_dim = df_train_3.shape[1]
    display(num_dim)
    df_train_3.columns = range(len(df_train_3.columns))

    train_2_sub = (train_2_normal['Subject'].unique()).tolist()

    m = 0
    for i in train_2_sub:
        train_2_normal_new = train_2_normal[(train_2_normal['Subject']== i)]
        df_row_count = train_2_normal_new.shape[0]
        if df_row_count < desired_length:
            rows_to_add = desired_length - df_row_count
            train_2_normal_new.tail()
            last_row = train_2_normal_new.ffill().iloc[[-1]]  # Extract the last row
            new_rows_df = pd.concat([last_row] * rows_to_add, ignore_index=True)
            train_2_normal_initial = pd.concat([train_2_normal_new, new_rows_df], ignore_index=True)
            train_2_normal_initial_no_sub = train_2_normal_initial.drop(columns='Subject')
            train_2_normal_final = z_normalize(train_2_normal_initial_no_sub)
            train_2_normal_final.insert(0, 'Subject', i)

        else:
            train_2_normal_initial = train_2_normal_new.iloc[:desired_length,:]
            train_2_normal_initial_no_sub = train_2_normal_initial.drop(columns='Subject')
            train_2_normal_final = z_normalize(train_2_normal_initial_no_sub)
            train_2_normal_final.insert(0, 'Subject', i)
            
        train_2_normal_final.columns = range(len(train_2_normal_final.columns))
        
        df_train_3.at[m,0] = i
        j = 0
        while j < num_dim:
            df_train_3.at[m,j] = train_2_normal_final[j]

            j += 1
        m += 1    

    df_train_3['class'] = 'normal'
    df_normal_baseline = df_train_3.iloc[:,1:]
    return(df_normal_baseline)

In [8]:
def preprocessing_dataframe(df,desired_length):
    train_1_abnormal = df.copy()
    df_train_2 = pd.DataFrame(columns = train_1_abnormal.columns)
    num_dim_ab = df_train_2.shape[1]

    df_train_2.columns = range(len(df_train_2.columns))

    train_1_ab_sub = (train_1_abnormal['Subject'].unique()).tolist()

    m = 0
    for i in train_1_ab_sub:
        train_1_abnormal_new = train_1_abnormal[(train_1_abnormal['Subject']== i)]
        df_row_count = train_1_abnormal_new.shape[0]
        if df_row_count < desired_length:
            rows_to_add = desired_length - df_row_count

            last_row = train_1_abnormal_new.ffill().iloc[[-1]]  # Extract the last row
            new_rows_df = pd.concat([last_row] * rows_to_add, ignore_index=True)
            train_1_abnormal_initial = pd.concat([train_1_abnormal_new, new_rows_df], ignore_index=True)
            train_1_abnormal_initial_no_sub = train_1_abnormal_initial.drop(columns='Subject')
            train_1_abnormal_final = z_normalize(train_1_abnormal_initial_no_sub)
            train_1_abnormal_final.insert(0, 'Subject', i)

        else:
            train_1_abnormal_initial = train_1_abnormal_new.iloc[:desired_length,:]
            train_1_abnormal_initial_no_sub = train_1_abnormal_initial.drop(columns='Subject')
            train_1_abnormal_final = z_normalize(train_1_abnormal_initial_no_sub)
            train_1_abnormal_final.insert(0, 'Subject', i)
        train_1_abnormal_final.columns = range(len(train_1_abnormal_final.columns))

        
        df_train_2.at[m,0] = i
        j = 0
        while j < num_dim_ab:
            df_train_2.at[m,j] = train_1_abnormal_final[j]

            j += 1
        m += 1    
    # display(df_train_2)
    df_train_2['class'] = 'abnormal'
    df_final = df_train_2.iloc[:,1:]
    return(df_final)

In [9]:
def ts_classifier(df_class_1):
    acc_mean_list = []
    prec_mean_list = []
    recall_mean_list = []
    f1_mean_list = []

    X = df_class_1.iloc[:,:-1]
    y = df_class_1.iloc[:,-1]

    for i in range(1, 31):
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=i, stratify=y)
        
        accuracy_dict = {} 
        precision_dict = {}
        recall_dict = {}
        f1_dict = {}
        
        comb = 1
        for a in[2,4,6]:
            for b in [4,8]:
                clf = MUSE(window_inc=a,alphabet_size=b,use_first_order_differences=False, random_state=42)
                clf.fit(X_train, y_train)
                
                # Predict using the final model
                y_pred = clf.predict(X_test)

                pos_label = 'abnormal'

                acc = accuracy_score(y_test, y_pred)
                prec = precision_score(y_test, y_pred, pos_label=pos_label)
                recall = recall_score(y_test, y_pred, pos_label=pos_label)
                f1 = f1_score(y_test, y_pred, pos_label=pos_label)
                
                accuracy_dict[(a, b)] = acc
                precision_dict[(a, b)] = prec
                recall_dict[(a, b)] = recall
                f1_dict[(a, b)] = f1
                
                display("Completed comb",comb)
                comb += 1
        
        accuracy_max_dict = max(accuracy_dict.items(), key=lambda x: x[1])
        display("Accuracy Dictionary:",accuracy_dict)
        accuracy_max = accuracy_max_dict[1]

        max_a_b = accuracy_max_dict[0]
        display("(a, b) combination for maximum accuracy:",max_a_b)

        display('Accuracy: %f' % accuracy_max)
        acc_mean_list.append(accuracy_max)

        precision_max = precision_dict[max_a_b]
        display('Precision: %f' % precision_max)
        prec_mean_list.append(precision_max)

        recall_max = recall_dict[max_a_b]
        display('Recall: %f' % recall_max)
        recall_mean_list.append(recall_max)

        f1_score_max = f1_dict[max_a_b]
        display('f1_score: %f' % f1_score_max)
        f1_mean_list.append(f1_score_max)

        display("Completed", i)

    acc_mean = np.mean(acc_mean_list)
    acc_std_dev = np.std(acc_mean_list)
    display('Accuracy Mean: %f' % acc_mean)

    prec_mean = np.mean(prec_mean_list)
    prec_std_dev = np.std(prec_mean_list)

    recall_mean = np.mean(recall_mean_list)
    recall_std_dev = np.std(recall_mean_list)

    f1_mean = np.mean(f1_mean_list)
    f1_std_dev = np.std(f1_mean_list)
    display('F1 Mean: %f' % f1_mean)

    return (acc_mean, acc_std_dev, prec_mean, prec_std_dev, recall_mean, recall_std_dev, f1_mean, f1_std_dev)

In [10]:
# Keys to combine
key_to_combine1 = 'sample_numbers_list_0.0_to_0.1'
key_to_combine2 = 'sample_numbers_list_0.1_to_0.2'
key_to_combine3 = 'sample_numbers_list_0.2_to_0.3'
# Create a new dictionary with combined values
hb_dict = {
    'hb_key': final_numbers_dictionary[key_to_combine1] + final_numbers_dictionary[key_to_combine2]
+ final_numbers_dictionary[key_to_combine3]}

{'hb_key': [42,
  45,
  11,
  21,
  47,
  49,
  12,
  70,
  13,
  38,
  20,
  31,
  44,
  57,
  58,
  36,
  2,
  6,
  62,
  26]}

In [11]:
classifier_dict_hb = {}
key = 'hb_key'
normal_sample_numbers = [] 
normal_sample_numbers.extend(hb_dict[key])
normal_sample_length = len(normal_sample_numbers)
display(normal_sample_length)

if len(normal_sample_numbers) <= abnormal_no:
    df_normal = final_normal_df[final_normal_df['Subject'].isin(normal_sample_numbers)]
    display(df_normal.head())

    random.seed(42)
    abnormal_sample_numbers = random.sample(abnormal_indices.tolist(), normal_sample_length)
    df_abnormal = final_abnormal_df[final_abnormal_df['Subject'].isin(abnormal_sample_numbers)]
    display(df_abnormal.head())

    desired_length_for_ts = time_series_length_finding(df_abnormal)

    df_normal_preprocessed = preprocessing_normal_df(df_normal,desired_length_for_ts)
    df_abnormal_preprocessed = preprocessing_dataframe(df_abnormal,desired_length_for_ts)

    df_classifier = (pd.concat([df_normal_preprocessed,df_abnormal_preprocessed]).reset_index()).drop(columns='index')
    acc_mean,acc_std_dev,prec_mean,prec_std_dev,recall_mean,recall_std_dev,f1_mean,f1_std_dev = ts_classifier(df_classifier)

    classifier_dict_hb[key] = {'acc_mean': acc_mean,'acc_std_dev': acc_std_dev,'prec_mean': prec_mean,'prec_std_dev': prec_std_dev,'recall_mean': recall_mean,'recall_std_dev': recall_std_dev,'f1_mean': f1_mean,'f1_std_dev': f1_std_dev}

else:
    df_abnormal = final_abnormal_df

    random.seed(42)
    random_sample_numbers = random.sample(normal_sample_numbers, abnormal_no)
    df_normal = final_normal_df[final_normal_df['Subject'].isin(random_sample_numbers)]
    display(df_normal)

    desired_length_for_ts = time_series_length_finding(df_abnormal)

    df_normal_preprocessed = preprocessing_normal_df(df_normal,desired_length_for_ts)
    df_abnormal_preprocessed = preprocessing_dataframe(df_abnormal,desired_length_for_ts)

    df_classifier = (pd.concat([df_normal_preprocessed,df_abnormal_preprocessed]).reset_index()).drop(columns='index')
    acc_mean,acc_std_dev,prec_mean,prec_std_dev,recall_mean,recall_std_dev,f1_mean,f1_std_dev = ts_classifier(df_classifier)

    classifier_dict_hb[key] = {'acc_mean': acc_mean,'acc_std_dev': acc_std_dev,'prec_mean': prec_mean,'prec_std_dev': prec_std_dev,'recall_mean': recall_mean,'recall_std_dev': recall_std_dev,'f1_mean': f1_mean,'f1_std_dev': f1_std_dev}

20

,vgrf_left_1,vgrf_left_2,vgrf_left_3,vgrf_left_4,vgrf_left_5,vgrf_left_6,vgrf_left_7,vgrf_left_8,vgrf_right_1,vgrf_right_2,vgrf_right_3,vgrf_right_4,vgrf_right_5,vgrf_right_6,vgrf_right_7,vgrf_right_8,vgrf_left_total,vgrf_right_total,Subject
1212,287.980,204.600,100.540,49.940,8.470,72.050,16.720,22.880,33.330,30.47,40.700,56.540,13.750,110.990,65.450,55.110,763.180,406.340,2
1213,309.507,243.133,101.508,62.260,9.724,76.065,15.488,20.944,15.202,11.44,17.193,23.166,4.675,69.245,40.458,35.354,838.629,216.733,2
1214,325.105,278.278,100.342,77.110,10.142,77.880,14.036,18.304,6.578,0.00,5.214,0.000,0.000,1.650,2.541,2.101,901.197,18.084,2
1215,295.449,266.970,99.385,88.836,11.187,96.239,18.414,22.000,4.994,0.00,2.838,0.000,0.000,0.000,1.012,0.000,898.480,8.844,2
1216,259.138,250.690,91.586,108.273,14.531,115.115,25.806,24.464,2.387,0.00,0.000,0.000,0.000,0.000,0.000,0.000,889.603,2.387,2


,vgrf_left_1,vgrf_left_2,vgrf_left_3,vgrf_left_4,vgrf_left_5,vgrf_left_6,vgrf_left_7,vgrf_left_8,vgrf_right_1,vgrf_right_2,vgrf_right_3,vgrf_right_4,vgrf_right_5,vgrf_right_6,vgrf_right_7,vgrf_right_8,vgrf_left_total,vgrf_right_total,Subject
3638,379.060,152.020,194.076667,46.676667,9.313333,33.183333,46.603333,11.990,0.0,4.180,0.0,12.540,18.333333,49.536667,81.216667,93.500,872.923333,259.306667,4
3639,390.291,184.426,213.455000,63.701000,21.010000,60.599000,76.362000,19.580,0.0,0.418,0.0,3.234,3.366000,13.794000,18.722000,13.651,1029.424000,53.185000,4
3640,292.897,159.698,188.870000,91.861000,50.094000,111.364000,151.019000,41.437,0.0,0.000,0.0,0.000,0.000000,3.223000,5.214000,0.000,1087.240000,8.437000,4
3641,136.301,85.756,90.002000,146.773000,85.877000,185.295000,242.814000,84.480,0.0,0.000,0.0,0.000,0.000000,0.000000,0.000000,0.000,1057.298000,0.000000,4
3642,23.540,15.576,12.540000,131.373000,95.546000,232.188000,326.557000,118.591,0.0,0.000,0.0,0.000,0.000000,0.000000,0.000000,0.000,955.911000,0.000000,4


'Avg length:'

1169.8

...

'f1_score: 1.000000'

'Completed'

30

'Accuracy Mean: 0.800000'

'F1 Mean: 0.794312'

In [12]:
classifier_dict_hb

{'hb_key': {'acc_mean': 0.8,
  'acc_std_dev': 0.15000000000000002,
  'prec_mean': 0.7927777777777778,
  'prec_std_dev': 0.15454133651343263,
  'recall_mean': 0.8166666666666667,
  'recall_std_dev': 0.21343747458109494,
  'f1_mean': 0.7943121693121694,
  'f1_std_dev': 0.1685895728700707}}

In [13]:
import pickle

# Specify the file path where you want to save the pickle file
pickle_file_path = 'Final_hb_gait_MUSE_normalized_cv.pickle'

# Save the dictionary to a pickle file
with open(pickle_file_path, 'wb') as file:
    pickle.dump(classifier_dict_hb, file)

print(f"Dictionary saved to {pickle_file_path}")

In [14]:
all_values = [value for sublist in final_numbers_dictionary.values() for value in sublist]
healthy_dict = {'healthy_key': all_values}

{'healthy_key': [42,
  45,
  11,
  21,
  47,
  49,
  12,
  70,
  13,
  38,
  20,
  31,
  44,
  57,
  58,
  36,
  2,
  6,
  62,
  26,
  5,
  17,
  37,
  56,
  18,
  3,
  8,
  43,
  52,
  40,
  71,
  14,
  72,
  46,
  27,
  1,
  9,
  23,
  15,
  4,
  32,
  33,
  59,
  25,
  54,
  67,
  7,
  16,
  48,
  53,
  63,
  19,
  64,
  10,
  30,
  34,
  61,
  28,
  60,
  41,
  24,
  51,
  29,
  50,
  68,
  22,
  55,
  39,
  65,
  66,
  69,
  35]}

In [15]:
classifier_dict_healthy = {}
key = 'healthy_key'
normal_sample_numbers = [] 
normal_sample_numbers.extend(healthy_dict[key])
normal_sample_length = len(normal_sample_numbers)
display(normal_sample_length)

if len(normal_sample_numbers) <= abnormal_no:
    df_normal = final_normal_df[final_normal_df['Subject'].isin(normal_sample_numbers)]
    display(df_normal.head())

    random.seed(42)
    abnormal_sample_numbers = random.sample(abnormal_indices.tolist(), normal_sample_length)
    df_abnormal = final_abnormal_df[final_abnormal_df['Subject'].isin(abnormal_sample_numbers)]
    display(df_abnormal.head())

    desired_length_for_ts = time_series_length_finding(df_abnormal)

    df_normal_preprocessed = preprocessing_normal_df(df_normal,desired_length_for_ts)
    df_abnormal_preprocessed = preprocessing_dataframe(df_abnormal,desired_length_for_ts)

    df_classifier = (pd.concat([df_normal_preprocessed,df_abnormal_preprocessed]).reset_index()).drop(columns='index')
    acc_mean,acc_std_dev,prec_mean,prec_std_dev,recall_mean,recall_std_dev,f1_mean,f1_std_dev = ts_classifier(df_classifier)

    classifier_dict_healthy[key] = {'acc_mean': acc_mean,'acc_std_dev': acc_std_dev,'prec_mean': prec_mean,'prec_std_dev': prec_std_dev,'recall_mean': recall_mean,'recall_std_dev': recall_std_dev,'f1_mean': f1_mean,'f1_std_dev': f1_std_dev}

else:
    df_abnormal = final_abnormal_df

    random.seed(42)
    random_sample_numbers = random.sample(normal_sample_numbers, abnormal_no)
    df_normal = final_normal_df[final_normal_df['Subject'].isin(random_sample_numbers)]
    display(df_normal)

    desired_length_for_ts = time_series_length_finding(df_abnormal)

    df_normal_preprocessed = preprocessing_normal_df(df_normal,desired_length_for_ts)
    df_abnormal_preprocessed = preprocessing_dataframe(df_abnormal,desired_length_for_ts)

    df_classifier = (pd.concat([df_normal_preprocessed,df_abnormal_preprocessed]).reset_index()).drop(columns='index')
    acc_mean,acc_std_dev,prec_mean,prec_std_dev,recall_mean,recall_std_dev,f1_mean,f1_std_dev = ts_classifier(df_classifier)

    classifier_dict_healthy[key] = {'acc_mean': acc_mean,'acc_std_dev': acc_std_dev,'prec_mean': prec_mean,'prec_std_dev': prec_std_dev,'recall_mean': recall_mean,'recall_std_dev': recall_std_dev,'f1_mean': f1_mean,'f1_std_dev': f1_std_dev}

72

,vgrf_left_1,vgrf_left_2,vgrf_left_3,vgrf_left_4,vgrf_left_5,vgrf_left_6,vgrf_left_7,vgrf_left_8,vgrf_right_1,vgrf_right_2,vgrf_right_3,vgrf_right_4,vgrf_right_5,vgrf_right_6,vgrf_right_7,vgrf_right_8,vgrf_left_total,vgrf_right_total,Subject
0,199.100,87.340,91.806,24.090,20.702,87.670,88.352,63.778,163.900,78.243,112.420,49.995,13.750,103.565,144.408,79.530,662.838,745.811,1
1,196.889,86.020,91.575,23.914,20.911,87.450,89.276,64.372,163.273,77.550,112.420,49.445,13.750,104.390,143.836,79.530,660.407,744.194,1
2,191.488,83.600,89.232,24.090,20.911,87.901,90.860,64.570,162.019,77.781,112.420,50.820,13.750,105.490,146.410,79.530,652.652,748.220,1
3,201.476,88.220,94.182,24.618,20.493,88.594,85.646,63.976,162.228,77.550,112.145,50.820,13.750,103.312,140.327,77.341,667.205,737.473,1
4,262.999,119.361,115.676,28.391,19.030,92.884,70.444,59.917,157.157,75.163,103.895,50.545,10.175,86.218,102.905,56.991,768.702,643.049,1


,vgrf_left_1,vgrf_left_2,vgrf_left_3,vgrf_left_4,vgrf_left_5,vgrf_left_6,vgrf_left_7,vgrf_left_8,vgrf_right_1,vgrf_right_2,vgrf_right_3,vgrf_right_4,vgrf_right_5,vgrf_right_6,vgrf_right_7,vgrf_right_8,vgrf_left_total,vgrf_right_total,Subject
0,62.458,165.143,129.239,88.231,32.659,71.005,95.238,39.600,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.0,683.573,0.000,1
1,28.083,110.077,82.115,106.755,39.831,107.096,148.456,57.728,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.0,680.141,0.000,1
2,4.807,20.603,13.079,84.799,21.252,150.832,201.322,86.779,85.855,28.809,42.801,0.000,0.462,0.000,0.000,0.0,583.473,157.927,1
3,0.000,3.047,0.704,4.400,3.410,58.333,79.123,153.065,112.211,125.180,148.588,9.097,13.365,0.000,4.961,0.0,302.082,413.402,1
4,0.000,0.000,0.000,0.000,0.308,0.000,2.233,29.579,132.033,154.528,174.559,27.852,35.519,4.301,18.843,0.0,32.120,547.635,1


'Avg length:'

1153.375

...

'f1_score: 0.750000'

'Completed'

30

'Accuracy Mean: 0.764368'

'F1 Mean: 0.754782'

In [16]:
classifier_dict_healthy

{'healthy_key': {'acc_mean': 0.7643678160919539,
  'acc_std_dev': 0.06425482693964911,
  'prec_mean': 0.7843752081252082,
  'prec_std_dev': 0.0842335001589424,
  'recall_mean': 0.7368253968253969,
  'recall_std_dev': 0.09359643297185086,
  'f1_mean': 0.7547815468081304,
  'f1_std_dev': 0.0659340834016769}}

In [17]:
import pickle

# Specify the file path where you want to save the pickle file
pickle_file_path = 'Final_healthy_gait_MUSE_normalized_cv.pickle'

# Save the dictionary to a pickle file
with open(pickle_file_path, 'wb') as file:
    pickle.dump(classifier_dict_healthy, file)

print(f"Dictionary saved to {pickle_file_path}")